## Price Data Scheduler

- Function Goal:
    * pull 48 hours of hourly price data once every hour 
- Tech:
    * python schedule library

In [9]:
# import dependencies
import pandas as pd
import numpy as np
import os
import requests
import json
import schedule
import time
import math
import datetime
from dotenv import load_dotenv


# import functions from other notebooks
from utils.utils import epoch_to_datetime, get_historical_prices
from utils.eval import get_cryptocompare_data, get_kucoin_data

ModuleNotFoundError: No module named 'utils.trade'

In [6]:
load_dotenv()
api_key=os.getenv('CRYPTO_COMPARE_API_KEY')

In [8]:
def fetch_sUSD_price(file_path):
    print('Fetching sUSD price data ...')            
    """
    fetch last 48 hours of hourly price data
    """
    currency = 'sUSD'
    base_currency='USD'
    start_date = datetime.datetime.today() – datetime.timedelta(days=2)
    end_date = time.time() #current time

    sUSD_historical_prices = get_historical_prices(currency,
                                              base_currency,
                                              start_date
                                              end_date,
                                              api_key)
    
    sUSD_historical_prices.to_csv('./export_to_csv/sUSD_historical_price')
    file_path = Path('../data/sUSD_historical_price')
    
    
    """
    read csv & convert epoch to date time
    """
    get_cryptocompare_data(file_path)
    
    print(f'Current {currency} price, in {base_currency}: ${df['close']} ')

def fetch_USDC_price(file_path):
    print('Fetching USDC price data ...')            
    """
    fetch last 48 hours of hourly price data
    """
    currency = 'USDC'
    base_currency='USD'
    start_date = datetime.datetime.today() – datetime.timedelta(days=2)
    end_date = time.time() #current time

    USDC_historical_prices = get_historical_prices(currency,
                                              base_currency,
                                              start_date
                                              end_date,
                                              api_key)

    USDC_historical_prices.to_csv('./export_to_csv/USDC_historical_price')
    file_path = Path('data/USDC_historical_price')

    """
    read csv & convert epoch to date time
    """
    get_cryptocompare_data(file_path)
    
    print(f'Current {currency} price, in {base_currency}: ${df['close']} ')


def fetch_DAI_price(file_path):
    print('Fetching DAI price data ...')            
    """
    fetch last 48 hours of hourly price data
    """
    currency = 'DAI'
    base_currency='USD'
    start_date = datetime.datetime.today() – datetime.timedelta(days=2)
    end_date = time.time() #current time

    DAI_historical_prices = get_historical_prices(currency,
                                              base_currency,
                                              start_date
                                              end_date,
                                              api_key)

    DAI_historical_prices.to_csv('./export_to_csv/DAI_historical_price')
    file_path = Path('data/DAI_historical_price')

    """
    read csv & convert epoch to date time
    """
    get_cryptocompare_data(file_path)
    
    print(f'Current {currency} price, in {base_currency}: ${df['close']} ')

def fetch_USDT_price(file_path):
    print('Fetching USDT price data ...')            
    """
    fetch last 48 hours of hourly price data
    """
    currency = 'USDT'
    base_currency='USD'
    start_date = datetime.datetime.today() – datetime.timedelta(days=2)
    end_date = time.time() #current time

    USDT_historical_prices = get_historical_prices(currency,
                                              base_currency,
                                              start_date
                                              end_date,
                                              api_key)

    USDT_historical_prices.to_csv('./export_to_csv/USDT_historical_prices')
    file_path = Path('../data/USDT_historical_prices')

    """
    read csv & convert epoch to date time
    """
    get_cryptocompare_data(file_path)
    
    print(f'Current {currency} price, in {base_currency}: ${df['close']} ')

def fetch_ETH_price(file_path, df):
    print('Fetching ETH price data ...')            
    """
    fetch last 48 hours of hourly price data
    """
    currency = 'ETH'
    base_currency='USD'
    start_date = datetime.datetime.today() – datetime.timedelta(days=2)
    end_date = time.time() #current time

    ETH_historical_prices = get_historical_prices(currency,
                                              base_currency,
                                              start_date
                                              end_date,
                                              api_key)

    ETH_historical_prices.to_csv('./export_to_csv/ETH_historical_prices')
    file_path = Path('../data/ETH_historical_prices')

    """
    read csv & convert epoch to date time
    """
    get_cryptocompare_data(file_path)
    
    print(f'Current {currency} price, in {base_currency}: ${df['close']} ')
    
    
def fetch_BTC_price(file_path):
    print('Fetching BTC price data ...')            
    """
    fetch last 48 hours of hourly price data
    """
    currency = 'BTC'
    base_currency='USD'
    start_date = datetime.datetime.today() – datetime.timedelta(days=2)
    end_date = time.time() #current time

    BTC_historical_prices = get_historical_prices(currency,
                                              base_currency,
                                              start_date
                                              end_date,
                                              api_key)
    
    """
    update csv file
    """

    BTC_historical_prices.to_csv('./export_to_csv/BTC_historical_prices')
    file_path = Path('../data/BTC_historical_prices')


    get_cryptocompare_data(file_path)
    
    print(f'Current {currency} price, in {base_currency}: ${df['close']} ')
    
    
"""
set all jobs to run every hour
"""
schedule.every(1).hour.do(fetch_sUSD_price)
schedule.every(1).hour.do(fetch_USDC_price)
schedule.every(1).hour.do(fetch_DAI_price)
schedule.every(1).hour.do(fetch_USDT_price)
schedule.every(1).hour.do(fetch_ETH_price)
schedule.every(1).hour.do(fetch_BTC_price)


while True:
    schedule.run_pending()
    time.sleep(1)

SyntaxError: invalid character in identifier (3500904028.py, line 8)